In [16]:
import pandas as pd
import ast  # For safely parsing "(4.0, 5.0)" as a Python tuple

def transform_fenez_log_to_structured_with_ranges(
    csv_input="output/assigned/assigned_fenez_params.csv",
    csv_output="output/assigned/structured_fenez_params.csv",
):
    """
    Reads the 'flat' fenestration/material CSV (with param_name & assigned_value),
    and outputs a 'structured' CSV that:

      - Merges final assigned value + min/max range into one row per parameter.
      - Does NOT skip params that have empty or None values.
      - Always includes a row for any param that appears in the CSV, even if
        there's no final value or no range.

    Final columns:
      ogc_fid, sub_key, eplus_object_type, eplus_object_name,
      param_name, param_value, param_min, param_max
    """

    df = pd.read_csv(csv_input)

    # We'll keep track of data in a nested dict:
    # final_dict[(ogc_fid, sub_key)] = {
    #   "obj_type": <str or None>,
    #   "obj_name": <str or None>,
    #   "params": {
    #       "Thickness": {
    #          "value": <final assigned value or None>,
    #          "min": <float or None>,
    #          "max": <float or None>
    #       },
    #       "Conductivity": {...},
    #       etc.
    #   }
    # }

    final_dict = {}

    def get_subdict(fid, s_key):
        """Helper to retrieve or create the dictionary entry for (fid, s_key)."""
        if (fid, s_key) not in final_dict:
            final_dict[(fid, s_key)] = {
                "obj_type": None,
                "obj_name": None,
                "params": {}
            }
        return final_dict[(fid, s_key)]

    for i, row in df.iterrows():
        # Must have these columns to proceed
        if "ogc_fid" not in row or "param_name" not in row or "assigned_value" not in row:
            continue

        ogc_fid = row["ogc_fid"]
        param_name = str(row["param_name"])
        assigned_value = row["assigned_value"]

        # We only transform if param_name starts with "fenez_"
        # (Otherwise, skip non-fenestration logs.)
        if not param_name.startswith("fenez_"):
            continue

        # Remove the "fenez_" prefix => e.g. "doors_opq.Thermal_Resistance_range"
        remainder = param_name[len("fenez_"):]  # e.g. "doors_opq.Thermal_Resistance_range"

        if "." in remainder:
            sub_key, field = remainder.split(".", 1)
        else:
            # e.g. "wwr", "roughness" => treat them as sub_key=that word, no field
            sub_key = remainder
            field = None

        # Retrieve or create sub-dict for (ogc_fid, sub_key)
        subd = get_subdict(ogc_fid, sub_key)

        # (A) If this row indicates the E+ object type
        if field == "obj_type":
            # e.g. assigned_value = "MATERIAL:NOMASS"
            subd["obj_type"] = assigned_value

        # (B) If this row indicates the E+ object name
        elif field == "Name":
            subd["obj_name"] = assigned_value

        # (C) If the field ends with "_range", parse as min/max
        elif field and field.endswith("_range"):
            # e.g. field="Thermal_Resistance_range"
            base_param = field.replace("_range", "")  # "Thermal_Resistance"

            if base_param not in subd["params"]:
                subd["params"][base_param] = {"value": None, "min": None, "max": None}

            # assigned_value might be something like "(4.0, 5.0)" or maybe "None"
            try:
                maybe_tuple = ast.literal_eval(str(assigned_value))
                if isinstance(maybe_tuple, (list, tuple)) and len(maybe_tuple) == 2:
                    min_val, max_val = maybe_tuple
                    subd["params"][base_param]["min"] = min_val
                    subd["params"][base_param]["max"] = max_val
            except:
                pass  # If parse fails or it's "None", we leave them as None

        # (D) If it's a 'normal' field => param_name like "Thermal_Resistance"
        else:
            # If there's no field, we might be dealing with e.g. "roughness" or "wwr"
            # which are top-level. We'll store them as a param anyway, in case you want them.
            if field is None:
                # e.g. sub_key="wwr", so param_name = sub_key
                # We'll store it under subd["params"][sub_key] to keep it consistent
                p_name = sub_key
                if p_name not in subd["params"]:
                    subd["params"][p_name] = {"value": None, "min": None, "max": None}
                subd["params"][p_name]["value"] = assigned_value
            else:
                # e.g. field="Thermal_Resistance", assigned_value=4.23
                if field not in subd["params"]:
                    subd["params"][field] = {"value": None, "min": None, "max": None}
                subd["params"][field]["value"] = assigned_value

    # Now finalize. We'll produce a row for **every** param in subd["params"], even if
    # param_value is None and min/max are None.

    structured_rows = []
    for (fid, s_key), info in final_dict.items():
        obj_type = info["obj_type"]
        obj_name = info["obj_name"]
        params = info["params"]  # a dict => param_name => { value, min, max }

        # If no params => possibly skip if you want. But let's create NO row or a placeholder?
        # We do NOT skip them if you want a row. But let's skip if sub_key has no params at all.
        if not params:
            # Possibly store a row for "sub_key with no params"? If you want that, do:
            # structured_rows.append({ "ogc_fid": fid, "sub_key": s_key, ... })
            # But typically we skip if no actual param.
            continue

        # For each param in "params"
        for p_name, pvals in params.items():
            param_value = pvals["value"]
            param_min = pvals["min"]
            param_max = pvals["max"]

            structured_rows.append({
                "ogc_fid": fid,
                "sub_key": s_key,
                "eplus_object_type": obj_type,
                "eplus_object_name": obj_name,
                "param_name": p_name,
                "param_value": param_value,  # might be None or empty
                "param_min": param_min,      # might be None
                "param_max": param_max       # might be None
            })

    # Convert to DataFrame
    df_out = pd.DataFrame(structured_rows)

    # (Optional) attempt to float-convert param_value, param_min, param_max
    def try_float(x):
        try:
            return float(x)
        except:
            return x

    for col in ["param_value", "param_min", "param_max"]:
        df_out[col] = df_out[col].apply(try_float)

    df_out.to_csv(csv_output, index=False)
    print(f"[transform_fenez_log_to_structured_with_ranges] => wrote: {csv_output}")


if __name__ == "__main__":
    transform_fenez_log_to_structured_with_ranges(
        csv_input="output/assigned/assigned_fenez_params.csv",
        csv_output="output/assigned/structured_fenez_params.csv"
    )


[transform_fenez_log_to_structured_with_ranges] => wrote: output/assigned/structured_fenez_params.csv


In [14]:
import pandas as pd

def flatten_ventilation_data(df_input, out_build_csv, out_zone_csv):
    """
    Takes a DataFrame like:
       ogc_fid | param_name        | assigned_value (dict)
    And splits it into two DataFrames:
      1) building-level (flattening 'building_params')
      2) zone-level (flattening 'zones').

    Then writes them to CSV (out_build_csv, out_zone_csv).
    """

    # Lists to accumulate rows before converting to DataFrame
    building_rows = []
    zone_rows = []

    for idx, row in df_input.iterrows():
        bldg_id = row["ogc_fid"]
        param_name = row["param_name"]
        assigned_val = row["assigned_value"]  # either dict of building_params or dict of zones

        if param_name == "building_params":
            # assigned_val is a dict with infiltration_base, infiltration_base_range, etc.
            for k, v in assigned_val.items():
                building_rows.append({
                    "ogc_fid": bldg_id,
                    "param_name": k,
                    "param_value": v
                })

        elif param_name == "zones":
            # assigned_val is a dict: { "Zone1": {...}, "Zone2": {...}, ... }
            for zone_name, zone_dict in assigned_val.items():
                for zparam_name, zparam_val in zone_dict.items():
                    zone_rows.append({
                        "ogc_fid": bldg_id,
                        "zone_name": zone_name,
                        "param_name": zparam_name,
                        "param_value": zparam_val
                    })

        else:
            # If there's something else, handle or skip
            pass

    # Convert to DataFrame
    df_build = pd.DataFrame(building_rows)
    df_zone = pd.DataFrame(zone_rows)

    # Optional: reorder columns, if you like
    df_build = df_build[["ogc_fid", "param_name", "param_value"]]
    df_zone = df_zone[["ogc_fid", "zone_name", "param_name", "param_value"]]

    # Write to CSV
    df_build.to_csv(out_build_csv, index=False)
    df_zone.to_csv(out_zone_csv, index=False)

    print(f"[INFO] Wrote building-level picks to {out_build_csv}")
    print(f"[INFO] Wrote zone-level picks to {out_zone_csv}")



df_assigned = pd.read_csv(r"D:\Documents\E_Plus_2030_py\output\assigned\assigned_ventilation.csv")

flatten_ventilation_data(
    df_input=df_assigned,
    out_build_csv="output/assigned_vent_building.csv",
    out_zone_csv="output/assigned_vent_zones.csv"
)


AttributeError: 'str' object has no attribute 'items'

In [15]:
df_assigned = pd.read_csv(r"D:\Documents\E_Plus_2030_py\output\assigned\assigned_ventilation.csv")
df_assigned

,ogc_fid,param_name,assigned_value
0,4136730,building_params,"{'infiltration_base': 1.2278853596915769, 'inf..."
1,4136730,zones,{'Zone1': {'infiltration_object_name': 'Infil_...
2,4136732,building_params,"{'infiltration_base': 0.6278853596915768, 'inf..."
3,4136732,zones,{'Zone1_FrontPerimeter': {'infiltration_object...
